<img src="http://mint-project.info/assets/images/mint-logo-vertical.png"  alt="Example" width="50" height="50" align="left" hspace="200" hspace="200">

# Querying the MINT execution database

This notebook demos how to query the MINT results database.

**Table of contents**
* [Navigating the database](#navigating)
    - [Execution metadata](#exe)
    - [Model execution outputs](#out)
* [Example Query](#example)

Import the packages and connect to the database.

In [4]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import sqlalchemy as sqla
import numpy as np
import csv
import json

In [8]:
with open('DATABASES.json') as json_file:
    DATABASES = json.load(json_file)

# choose the database to use
db = DATABASES['devingestion']

# construct an engine connection string
engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = db['USER'],
    password = db['PASSWORD'],
    host = db['HOST'],
    port = db['PORT'],
    database = db['NAME'],
)

In [9]:
# create sqlalchemy engine
engine = create_engine(engine_string)
connection = engine.connect()
metadata = sqla.MetaData()

## <a name='navigating'> Navigating the database </a>

Below is a list of currently available tables in the database:

In [4]:
table_names = engine.table_names()
for item in table_names:
    print(item)

threads
threads_inputs
threads_input_column
cycles_0_9_4_alpha_advanced_pongo_weather_cycles_season
threads_output_table
cycles_0_9_4_alpha_advanced_pongo_weather_runs
cycles_0_9_4_alpha_advanced_pongo_cycles_season
cycles_0_9_4_alpha_advanced_pongo_runs
cycles_0_9_4_alpha_cycles_season
modflow_2005_cfg_runs
pihm_v4_1_0_runs
cycles_0_9_4_alpha_runs
economicgambella_v6_1_economic_land_use
economicgambella_v6_1_runs
pihm_v4_1_0_surf_images
pihm_v4_1_0_gw_images
hand_v1_simple_hand_image
hand_v1_simple_runs
fsi_1_0_simple_fsi_movie
fsi_1_0_simple_runs
cycles_0_9_4_alpha_cycles_weatheroutput
cycles_0_9_4_alpha_cycles_summary
cycles_0_9_4_alpha_cycles_soilprofile
cycles_0_9_4_alpha_cycles_nitrogen
cycles_0_9_4_alpha_cycles_water
cycles_0_9_4_alpha_cycles_crop
cycles_0_9_4_alpha_cycles_som
topoflow_cfg_simple_runs
topoflow_cfg_simple_topo_movie
cycles_0_9_4_alpha_collection_cycles_season
cycles_0_9_4_alpha_collection_runs
topoflow36_2_1_0_topo_movie
topoflow36_2_1_0_runs
economic_v7_economic

### <a name='exe'> Execution metadata </a>

The threads, threads_inputs, thread_input_column, and threads_output_table contains general information about the execution, organized as modeling thread. The tables are related using the threadid as the key.

In MINT, a modeling thread is used to explore different initial conditions, models, input data, or interventions. Each thread can consists of several executions. For instance, a point model executed at different location would result in multiple execution (and therefore mint_runid). Thread IDs are unique and can be used to relate provenance information, inputs and outputs table automatically. 

#### Threads table

This table contains the information about provenance, which describes the execution traces of the executions in MINT. The metadata associated with the provenance trace is given in a nested dictionaries. 

Let's print out the available column and dump the first 10 elements of the table into a dataframe.

In [5]:
inspector = inspect(engine)
z = inspector.get_columns('threads')
for item in z:
    print(item['name'])

threadid
metadata
category


In [6]:
sql_DF = pd.read_sql("SELECT * FROM threads limit 10",
                     con=engine)
sql_DF

,threadid,metadata,category
0,VggqferoeUnXQB93yeBM,"{'thread': {'id': 'VggqferoeUnXQB93yeBM', 'tex...",None
1,etZK2mpavnnjJRyfe5mm,"{'thread': {'id': 'etZK2mpavnnjJRyfe5mm', 'tex...",None
2,l3EAhLLCUv2NtFbYVLkp,"{'thread': {'id': 'l3EAhLLCUv2NtFbYVLkp', 'tex...",None
3,53CKtJZb6U8rwdjJF4f6,"{'thread': {'id': '53CKtJZb6U8rwdjJF4f6', 'tex...",None
4,YgXRKKgAjCixdcbVUOar,"{'thread': {'id': 'YgXRKKgAjCixdcbVUOar', 'tex...",None
5,vk5GDQ6Y7Xd1jRIyTgTx,"{'thread': {'id': 'vk5GDQ6Y7Xd1jRIyTgTx', 'tex...",None
6,Ghpr0RMeByd7vCiJ9O5R,"{'thread': {'id': 'Ghpr0RMeByd7vCiJ9O5R', 'tex...",None
7,uZWdcJMRjwmVm8FdIU7i,"{'thread': {'id': 'uZWdcJMRjwmVm8FdIU7i', 'tex...",None
8,zeb8nvpXeiQ5sQVxKt4I,"{'thread': {'id': 'zeb8nvpXeiQ5sQVxKt4I', 'tex...",None
9,DKn6afMctvVQhaOg0Y5Q,"{'thread': {'id': 'DKn6afMctvVQhaOg0Y5Q', 'tex...",None


The metadata column constains the provenance information for a specific thread. Let's look at the second row:

In [7]:
sql_DF['metadata'].iloc[1]

{'thread': {'id': 'etZK2mpavnnjJRyfe5mm',
  'text': 'Guder Region',
  'models': {'https://w3id.org/okn/i/mint/pihm-v4.1.0-guder': {'id': 'https://w3id.org/okn/i/mint/pihm-v4.1.0-guder',
    'name': 'PIHM++ v4 configuration (v4) for the Guder region in Ethiopia with 1 year of GLDAS data (2017)',
    'wcm_uri': 'https://github.com/mintproject/MINT-WorkflowDomain/raw/master/WINGSWorkflowComponents/pihm-4.1.0/pihm-4.1.0.zip',
    'category': 'Hydrology',
    'localname': 'pihm-v4.1.0-guder',
    'model_type': '',
    'description': 'The Penn State Integrated Hydrologic Modeling System (PIHM) is a finite volume code used for simulating the distributed hydrologic states of a given watershed or river basin. PIHM accounts for many physical processes including: land surface processes (evaporation- transpiration, canopy interception,  snowmelt) overland/channel flow coupled to groundwater flow. PIHM can include reservoirs and flow control structures. PIHM applies adaptive time stepping and uses 

In this example, the thread corresponds to a run with the PIHM hydrology model. The provenance keeps a record of some of the information contained in the model catalog about the particuliar  setup used for execution, input parameters for the run, the tasks the modeler was trying to accomplish, the location and time frame. 

In MINT, tasks are used to answer a specific problem. For instance, understanding food security for a sepcific region might require some knowledge of flooding during the growing season and the use of a hydrology model. Note that the time frame of the tasks does not necessarily reflect that of the problem statement. In the first example, flooding is relevant to both the planting time and growing season of an agriculture model which would place the start of the simulation earlier than the problem’s time frame.

#### Threads_inputs

The table includes information about the input files, start and end date of the simulation, and the geospatial extent.

In [8]:
inspector = inspect(engine)
z1 = inspector.get_columns('threads_inputs')
headers_inputs = []
for item in z1:
    headers_inputs.append(item['name'])
    print(item['name'])

threadid
id
url
start_date
end_date
spatial_type
x
y
x_max
x_min
y_max
y_min


In [9]:
sql_DF_threads_inputs = pd.read_sql("SELECT * FROM threads_inputs limit 10",
                     con=engine)
sql_DF_threads_inputs

,threadid,id,url,start_date,end_date,spatial_type,x,y,x_max,x_min,y_max,y_min
0,zLQnesC9wps2h5ZdYpMi,crops.crop,https://publisher.mint.isi.edu/i5gwq/crops.crop,None,None,None,NaN,NaN,None,None,None,None
1,zLQnesC9wps2h5ZdYpMi,pongo.soil,https://publisher.mint.isi.edu/OIvRW/pongo.soil,None,None,None,NaN,NaN,None,None,None,None
2,zLQnesC9wps2h5ZdYpMi,met9.12Nx27.62E.weather,https://publisher.mint.isi.edu/kkA2u/met9.12Nx...,None,None,Point,27.62,9.12,None,None,None,None
3,gwO5l7SeXmyyKWFzv9X0,cyclesdata.csv,https://publisher.mint.isi.edu/eTdtX/cyclesdat...,None,None,None,NaN,NaN,None,None,None,None
4,gwO5l7SeXmyyKWFzv9X0,calibdata.csv,https://publisher.mint.isi.edu/14mJfs/calibdat...,None,None,None,NaN,NaN,None,None,None,None
5,gwO5l7SeXmyyKWFzv9X0,pongo-price.csv,https://data.mint.isi.edu/files/economic/pongo...,None,None,None,NaN,NaN,None,None,None,None
6,gwO5l7SeXmyyKWFzv9X0,pongo-productioncost.csv,https://data.mint.isi.edu/files/economic/pongo...,None,None,None,NaN,NaN,None,None,None,None
7,BLKIjJato4SUDJAeWmBg,crops.crop,https://publisher.mint.isi.edu/i5gwq/crops.crop,None,None,None,NaN,NaN,None,None,None,None
8,BLKIjJato4SUDJAeWmBg,pongo.soil,https://publisher.mint.isi.edu/OIvRW/pongo.soil,None,None,None,NaN,NaN,None,None,None,None
9,BLKIjJato4SUDJAeWmBg,met9.12Nx27.62E.weather,https://publisher.mint.isi.edu/kkA2u/met9.12Nx...,None,None,Point,27.62,9.12,None,None,None,None


#### Thread_output_table

This table relates the thread to the output table(s) for the model execution as well as the date of the ingestion.

In [10]:
inspector = inspect(engine)
z2 = inspector.get_columns('threads_output_table')
for item in z2:
    print(item['name'])

threadid
modelid
table_schema
output_table_name
ingestion_date


In [11]:
sql_DF_threads_output_table = pd.read_sql("SELECT * FROM threads_output_table limit 10",
                     con=engine)
sql_DF_threads_output_table

,threadid,modelid,table_schema,output_table_name,ingestion_date
0,PvjIvVMa4ELIQQcDYgIM,https://w3id.org/okn/i/mint/pihm-v4.1.0-beko-t...,public,pihm_v4_1_0_surf_images,2019-11-26 13:43:50.404903
1,53CKtJZb6U8rwdjJF4f6,https://w3id.org/okn/i/mint/cycles-0.9.4-alpha...,public,cycles_0_9_4_alpha_advanced_pongo_cycles_season,2019-10-18 23:30:02.060834
2,YgXRKKgAjCixdcbVUOar,https://w3id.org/okn/i/mint/cycles-0.9.4-alpha...,public,cycles_0_9_4_alpha_advanced_pongo_weather_cycl...,2019-10-18 23:36:11.789881
3,vk5GDQ6Y7Xd1jRIyTgTx,https://w3id.org/okn/i/mint/economic-v6.1_adva...,public,economic_v6_1_advanced_pongo_economic_land_use,2019-10-20 22:33:46.079579
4,Ghpr0RMeByd7vCiJ9O5R,https://w3id.org/okn/i/mint/economic-v6.1_adva...,public,economic_v6_1_advanced_pongo_economic_land_use,2019-10-20 22:38:52.232468
5,b2oR7iGkFEzVgimbNZFO,https://w3id.org/okn/i/mint/cycles-0.9.4-alpha...,public,cycles_0_9_4_alpha_cycles_season,2019-10-21 20:13:30.580806
6,zLQnesC9wps2h5ZdYpMi,https://w3id.org/okn/i/mint/cycles-0.9.4-alpha...,public,cycles_0_9_4_alpha_cycles_season,2019-10-22 01:52:50.193797
7,gwO5l7SeXmyyKWFzv9X0,https://w3id.org/okn/i/mint/economic-v6.1_adva...,public,economic_v6_1_economic_land_use,2019-10-22 01:39:24.532745
8,F9odlTVcnDy0WPDQkKi2,https://w3id.org/okn/i/mint/cycles-0.9.4-alpha...,public,cycles_0_9_4_alpha_cycles_season,2019-10-22 01:46:17.761295
9,BLKIjJato4SUDJAeWmBg,https://w3id.org/okn/i/mint/cycles-0.9.4-alpha...,public,cycles_0_9_4_alpha_cycles_season,2019-10-22 02:02:03.237047


### <a name='out'> Model  execution outputs </a>

Tables are generated automatically for each model version (if not already in the database), one table per input and output in the model. 

The naming convention is as follows: 
* output data: modelName_version_fileName
* input data: modelName_version_runs

Each table contains a mint_runid (the execution ID), which is usnique for each model execution and can be used as a key to relate inputs and outputs tables. The threadID can be used to relate model outputs to thread metadata. Since multiple executions can happen in a thread, the threadID is not unique in these tables. The other columns correspond to output variables for the model and differs from table to table.

Let's inspect the input and output tables (season, where most of the relevant outputs are) for Cycles.

In [12]:
#Inspect headers of the Cycles v0.9.4 input tables
v = inspector.get_columns('cycles_0_9_4_alpha_runs')
headers_in =[]
for item in v:
    headers_in.append(item['name'])
    print(item['name'])

threadid
mint_runid
cycles_crops
cycles_soil
cycles_weather
crop_name
end_planting_day
end_year
fertilizer_rate
start_planting_day
start_year
use_forcing
weed_fraction


In [13]:
sql_DF_cycles_input = pd.read_sql("SELECT * FROM cycles_0_9_4_alpha_runs limit 10",
                     con=engine)
sql_DF_cycles_input

,threadid,mint_runid,cycles_crops,cycles_soil,cycles_weather,crop_name,end_planting_day,end_year,fertilizer_rate,start_planting_day,start_year,use_forcing,weed_fraction
0,b2oR7iGkFEzVgimbNZFO,f36b9357b53915411cb44b64a66daae3,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,114,2000,False,1.50
1,b2oR7iGkFEzVgimbNZFO,e00fa78d41e8b954e25466cf443c579e,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,114,2000,False,0.40
2,b2oR7iGkFEzVgimbNZFO,1527486e2a7432581b768d79a2514757,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,114,2000,False,0.20
3,b2oR7iGkFEzVgimbNZFO,edaf6b771c8652638785d3f2c5505ce2,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,114,2000,False,0.10
4,b2oR7iGkFEzVgimbNZFO,33f342f3dac6a7b33fa2bbe3bed33149,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,114,2000,False,0.05
5,b2oR7iGkFEzVgimbNZFO,08829f42294e393b68d7ac557ed2db5d,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,114,2000,False,0.00
6,b2oR7iGkFEzVgimbNZFO,b2d1d61eb745ca8dfb92b7fe5681ed22,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,100,2000,False,1.50
7,b2oR7iGkFEzVgimbNZFO,f1ccf9ee5e307c6edce2f53263f4c4d0,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,100,2000,False,0.40
8,b2oR7iGkFEzVgimbNZFO,a641ba83e87b94e238c5d4636bb66cd2,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,100,2000,False,0.20
9,b2oR7iGkFEzVgimbNZFO,4bb31e66ee86769c5155072f21e72783,crops.crop,pongo.soil,000a3137-d9c2-5ab4-9f2f-b80f4257c063,Peanut,149,2017,625.0,100,2000,False,0.10


In [14]:
#Inspect headers of the Cycles v0.9.4 output table for the season file
w = inspector.get_columns('cycles_0_9_4_alpha_cycles_season')
headers_out = []
for item in w:
    headers_out.append(item['name'])
    print(item['name'])

threadid
mint_runid
date
crop
total_biomass
root_biomass
grain_yield
forage_yield
ag_residue
harvest_index
potential_tr
actual_tr
soil_evap
total_n
root_n
grain_n
forage_n
cum__n_stress
n_in_harvest
n_in_residue
n_concn_forage


In [15]:
sql_DF_cycles_output = pd.read_sql("SELECT * FROM cycles_0_9_4_alpha_cycles_season limit 10",
                     con=engine)
sql_DF_cycles_output

,threadid,mint_runid,date,crop,total_biomass,root_biomass,grain_yield,forage_yield,ag_residue,harvest_index,...,actual_tr,soil_evap,total_n,root_n,grain_n,forage_n,cum__n_stress,n_in_harvest,n_in_residue,n_concn_forage
0,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2006-08-15,Peanut,6.507528,0.800251,1.367745,3.471625,0.867906,0.239649,...,199.631014,211.291937,0.150591,0.007979,0.057522,0.068072,0.0,125.594105,24.996698,1.960804
1,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2007-07-31,Peanut,7.455461,0.943260,1.556306,3.964716,0.991179,0.238983,...,239.227066,163.664792,0.164474,0.009412,0.062371,0.074152,0.0,136.523554,27.950411,1.870306
2,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2008-08-01,Peanut,7.248837,0.911916,1.507459,3.863570,0.965893,0.237885,...,227.593723,166.372500,0.162128,0.009101,0.061271,0.073404,0.0,134.675318,27.452254,1.899901
3,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2009-08-03,Peanut,5.343741,0.646500,1.145029,2.841769,0.710442,0.243766,...,188.054446,201.313468,0.123415,0.006447,0.047980,0.055191,0.0,103.170823,20.244674,1.942135
4,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2010-08-17,Peanut,7.379032,0.915628,1.539761,3.938915,0.984729,0.238227,...,224.079997,148.964033,0.164177,0.009127,0.062171,0.074304,0.0,136.474562,27.702638,1.886403
5,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2011-08-28,Peanut,9.203341,1.107361,1.952527,4.914763,1.228691,0.241172,...,309.513587,165.271782,0.188044,0.011043,0.071842,0.084127,0.0,155.968999,32.074582,1.711713
6,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2012-09-03,Peanut,8.200779,1.015952,1.709658,4.380136,1.095034,0.237954,...,239.971354,175.131616,0.173353,0.010129,0.065373,0.078281,0.0,143.654018,29.699189,1.787175
7,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2013-08-20,Peanut,8.681602,1.095354,1.815442,4.616645,1.154161,0.239307,...,294.952047,159.760081,0.179440,0.010931,0.067871,0.080510,0.0,148.380903,31.058685,1.743901
8,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2014-07-31,Peanut,7.463221,0.884121,1.586870,3.993784,0.998446,0.241199,...,275.901082,193.536196,0.161309,0.008822,0.061899,0.072470,0.0,134.369502,26.939957,1.814575
9,b2oR7iGkFEzVgimbNZFO,04d5863a5f8e630318e094b89eb8e9d8,2015-08-09,Peanut,7.393377,0.902691,1.549261,3.953139,0.988285,0.238690,...,301.056371,160.469367,0.162750,0.009000,0.061768,0.073585,0.0,135.353610,27.395921,1.861441


If a model output is in binary format (e.g., netcdf) or a movie, a link to the aggregated datafiles will be provided. For instance, let's consider the outputs for the Topoflow hydrology model:

In [16]:
#Inspect headers of the topoflow36_2_1_0_topo_movie
w = inspector.get_columns('topoflow36_2_1_0_topo_movie')
for item in w:
    print(item['name'])

threadid
mint_runid
url


In [17]:
sql_DF_topo_movie = pd.read_sql("SELECT * FROM topoflow36_2_1_0_topo_movie limit 10",
                     con=engine)
sql_DF_topo_movie

,threadid,mint_runid,url
0,3jIm5oOQscQ14Tj3SM1b,d0330c8c89982c78554eec38ba5a6773,https://ingestion.mint.isi.edu/data/aggregated...
1,3jIm5oOQscQ14Tj3SM1b,9a94a97dfb11f5e76d7c26fcd12719d3,https://ingestion.mint.isi.edu/data/aggregated...
2,U2lQrX3YHzwPL213BNCZ,70d36ee1e1361fc01cae36d1b5956d61,https://ingestion.mint.isi.edu/data/aggregated...
3,U2lQrX3YHzwPL213BNCZ,078211d32c267277358cba7898f797cd,https://ingestion.mint.isi.edu/data/aggregated...
4,cIbXu5eRJFae433i6uGa,0b25c77956458a3c2dce612af56b7419,https://ingestion.mint.isi.edu/data/aggregated...
5,cIbXu5eRJFae433i6uGa,fdb2c93fa7aa23f614c4903554618e92,https://ingestion.mint.isi.edu/data/aggregated...
6,LFb7bRAg3okZWpYtTM14,b0da961690a112094c4a0028bb6b8ef9,https://ingestion.mint.isi.edu/data/aggregated...
7,LFb7bRAg3okZWpYtTM14,25c7250ef0b5517ffbbde264250b5410,https://ingestion.mint.isi.edu/data/aggregated...
8,6zbIV0OSUxxbBoOlPufe,31a23bd7171e01c521ac4230c6e0578f,https://ingestion.mint.isi.edu/data/aggregated...
9,7WuXvRWbasOOup2fAWEn,2683115789802e70781a725fa4c42f0b,https://ingestion.mint.isi.edu/data/aggregated...


## <a name=example> Example Query </a>

In this example, we want to query all the Cycles executions for the year 2017, with fertilizer_rate of 100, and 'Maize'

In [18]:
cycles_input = sqla.Table('cycles_0_9_4_alpha_runs', metadata, autoload = True, autoload_with = engine)
cycles_output = sqla.Table('cycles_0_9_4_alpha_cycles_season', metadata, autoload = True, autoload_with = engine)

In [19]:
ci = sqla.select([cycles_input]).where((cycles_input.columns['crop_name'] == 'Maize') &\
                                     (cycles_input.columns['fertilizer_rate'] == 100) &\
                                     (cycles_input.columns['start_year'] == 2017) &\
                                     (cycles_input.columns['end_year'] == 2017))

Query the database and fetch the results in a dataframe

In [20]:
ResultProxy = connection.execute(ci)
inputs = ResultProxy.fetchall()
df_in = pd.DataFrame(inputs, columns=headers_in)

In [21]:
df_in

,threadid,mint_runid,cycles_crops,cycles_soil,cycles_weather,crop_name,end_planting_day,end_year,fertilizer_rate,start_planting_day,start_year,use_forcing,weed_fraction
0,BLKIjJato4SUDJAeWmBg,6bd197ad3ecf0d96ab05a1045560353a,crops.crop,pongo.soil,met9.12Nx27.62E.weather,Maize,149,2017,100.0,100,2017,False,0.0
1,BLKIjJato4SUDJAeWmBg,5fb5a46259cbbb1a709f12e3298abf11,crops.crop,pongo.soil,met9.12Nx27.62E.weather,Maize,149,2017,100.0,100,2017,False,0.1
2,BLKIjJato4SUDJAeWmBg,159e26340a0955f6e1f2cfe203704a82,crops.crop,pongo.soil,met9.12Nx27.62E.weather,Maize,149,2017,100.0,100,2017,False,0.2
3,BLKIjJato4SUDJAeWmBg,c75062df54b9bedff4ecc017b612486e,crops.crop,pongo.soil,met9.12Nx27.62E.weather,Maize,149,2017,100.0,100,2017,False,0.3
4,BLKIjJato4SUDJAeWmBg,149e95edaae1d65bb8389bd4f3620988,crops.crop,pongo.soil,met9.12Nx27.62E.weather,Maize,149,2017,100.0,100,2017,False,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,KRcxi4RyIhK3CBkNf4eH,decfe04128936fc22eda81f409cdcfb8,crops.crop,pongo.soil,bb980661-a48f-588e-8111-1b354b1444d9,Maize,149,2017,100.0,100,2017,False,0.4
176,KRcxi4RyIhK3CBkNf4eH,6b7516bfab652e1b6bf602d77e2f1e3b,crops.crop,pongo.soil,bb980661-a48f-588e-8111-1b354b1444d9,Maize,149,2017,100.0,100,2017,False,0.3
177,KRcxi4RyIhK3CBkNf4eH,839459c071a7a0ae4dc4b367fedb5275,crops.crop,pongo.soil,bb980661-a48f-588e-8111-1b354b1444d9,Maize,149,2017,100.0,100,2017,False,0.2
178,KRcxi4RyIhK3CBkNf4eH,94d8908dc1d9224a597c25fce2d46f9d,crops.crop,pongo.soil,bb980661-a48f-588e-8111-1b354b1444d9,Maize,149,2017,100.0,100,2017,False,0.1


Next retrieve the corresponding grain_yield in the cycles season output table:

In [22]:
df_out = pd.DataFrame(columns=headers_out)
for row in df_in.iterrows():
    ID  = row[1][1]
    #print(ID)
    cout = sqla.select([cycles_output]).where(cycles_output.columns['mint_runid'] == ID)
    ResultProxy1 = connection.execute(cout)
    outputs = ResultProxy1.fetchall()
    df2 = pd.DataFrame(outputs,columns=headers_out)
    df_out = df_out.append(df2,ignore_index=True)
df_out

,threadid,mint_runid,date,crop,total_biomass,root_biomass,grain_yield,forage_yield,ag_residue,harvest_index,...,actual_tr,soil_evap,total_n,root_n,grain_n,forage_n,cum__n_stress,n_in_harvest,n_in_residue,n_concn_forage
0,BLKIjJato4SUDJAeWmBg,6bd197ad3ecf0d96ab05a1045560353a,2017-10-04,Maize,10.214418,1.509399,3.433010,0.000000,5.272009,0.394371,...,318.693933,164.598621,0.066482,0.007366,0.036361,0.000000,0.269055,36.361172,30.120904,0.000000
1,VVWwaIZQCvfr0z588tZF,6bd197ad3ecf0d96ab05a1045560353a,2017-10-04,Maize,10.214418,1.509399,3.433010,0.000000,5.272009,0.394371,...,318.693933,164.598621,0.066482,0.007366,0.036361,0.000000,0.269055,36.361172,30.120904,0.000000
2,BLKIjJato4SUDJAeWmBg,5fb5a46259cbbb1a709f12e3298abf11,2017-10-04,Maize,9.677826,1.432804,3.240053,0.000000,5.004969,0.392971,...,307.571706,176.166607,0.063295,0.006972,0.034560,0.000000,0.276173,34.560293,28.735010,0.000000
3,VVWwaIZQCvfr0z588tZF,5fb5a46259cbbb1a709f12e3298abf11,2017-10-04,Maize,9.677826,1.432804,3.240053,0.000000,5.004969,0.392971,...,307.571706,176.166607,0.063295,0.006972,0.034560,0.000000,0.276173,34.560293,28.735010,0.000000
4,BLKIjJato4SUDJAeWmBg,159e26340a0955f6e1f2cfe203704a82,2017-10-04,Maize,9.640192,1.426526,3.226461,0.000000,4.987204,0.392816,...,305.830259,181.930747,0.063090,0.006960,0.034432,0.000000,0.275102,34.432484,28.657775,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,KRcxi4RyIhK3CBkNf4eH,decfe04128936fc22eda81f409cdcfb8,2017-10-16,Maize,11.319298,1.706856,3.812261,0.000000,5.800181,0.396597,...,385.742751,178.956422,0.069378,0.007932,0.037939,0.000000,0.326161,37.938654,31.439590,0.000000
186,KRcxi4RyIhK3CBkNf4eH,6b7516bfab652e1b6bf602d77e2f1e3b,2017-10-16,Maize,11.348952,1.709833,3.812944,0.000000,5.826175,0.395570,...,388.703606,174.042872,0.069523,0.007933,0.037961,0.000000,0.327927,37.960925,31.562177,0.000000
187,KRcxi4RyIhK3CBkNf4eH,839459c071a7a0ae4dc4b367fedb5275,2017-10-16,Maize,11.309986,1.705698,3.792801,0.000000,5.811487,0.394907,...,390.405552,168.967915,0.069196,0.007883,0.037747,0.000000,0.330701,37.747204,31.448929,0.000000
188,KRcxi4RyIhK3CBkNf4eH,94d8908dc1d9224a597c25fce2d46f9d,2017-10-16,Maize,11.296354,1.703323,3.784588,0.000000,5.808444,0.394514,...,391.950980,162.865700,0.069171,0.007847,0.037729,0.000000,0.332401,37.729070,31.442372,0.000000


To get the corresponding lat/lon:

In [23]:
threads_inputs = sqla.Table('threads_inputs', metadata, autoload = True, autoload_with = engine)
df_loc = pd.DataFrame(columns=headers_inputs)
for row in df_in.iterrows():
    ID  = row[1][0]
    #print(ID)
    cout = sqla.select([threads_inputs]).where(threads_inputs.columns['threadid'] == ID)
    ResultProxy1 = connection.execute(cout)
    outputs = ResultProxy1.fetchall()
    df2 = pd.DataFrame(outputs,columns=headers_inputs)
    df_loc = df_out.append(df2,ignore_index=True)
df_loc

,threadid,mint_runid,date,crop,total_biomass,root_biomass,grain_yield,forage_yield,ag_residue,harvest_index,...,url,start_date,end_date,spatial_type,x,y,x_max,x_min,y_max,y_min
0,BLKIjJato4SUDJAeWmBg,6bd197ad3ecf0d96ab05a1045560353a,2017-10-04,Maize,10.214418,1.509399,3.433010,0.000000,5.272009,0.394371,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VVWwaIZQCvfr0z588tZF,6bd197ad3ecf0d96ab05a1045560353a,2017-10-04,Maize,10.214418,1.509399,3.433010,0.000000,5.272009,0.394371,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BLKIjJato4SUDJAeWmBg,5fb5a46259cbbb1a709f12e3298abf11,2017-10-04,Maize,9.677826,1.432804,3.240053,0.000000,5.004969,0.392971,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VVWwaIZQCvfr0z588tZF,5fb5a46259cbbb1a709f12e3298abf11,2017-10-04,Maize,9.677826,1.432804,3.240053,0.000000,5.004969,0.392971,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BLKIjJato4SUDJAeWmBg,159e26340a0955f6e1f2cfe203704a82,2017-10-04,Maize,9.640192,1.426526,3.226461,0.000000,4.987204,0.392816,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,KRcxi4RyIhK3CBkNf4eH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://data.mint.isi.edu/files/simulation-run...,2000-01-01 08:00:00+00:00,2018-01-01 07:59:59+00:00,Point,25.12,9.12,None,None,None,None
222,KRcxi4RyIhK3CBkNf4eH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://data.mint.isi.edu/files/simulation-run...,2000-01-01 08:00:00+00:00,2018-01-01 07:59:59+00:00,Point,25.62,8.12,None,None,None,None
223,KRcxi4RyIhK3CBkNf4eH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://data.mint.isi.edu/files/simulation-run...,2000-01-01 08:00:00+00:00,2018-01-01 07:59:59+00:00,Point,27.62,8.62,None,None,None,None
224,KRcxi4RyIhK3CBkNf4eH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://data.mint.isi.edu/files/simulation-run...,2000-01-01 08:00:00+00:00,2018-01-01 07:59:59+00:00,Point,25.62,7.62,None,None,None,None
